<center><h1>Homework 2</h1>
    <h3>Rebecca Hinrichs</h3>
    <h4>MATH 4301 Spring 2023</h4></center>

---

In [58]:
# Import libraries & dependencies
import math
import numpy as np
import scipy as sp
from numpy import array, zeros, diag, sqrt
from numpy.linalg import norm
from numpy.random import rand
from scipy.sparse import diags
import warnings
warnings.filterwarnings("ignore")  # suppress ill-conditioned warnings
np.set_printoptions(precision = 4)

<center><br>Questions are from <b>Problem Set 2.3</b> in <br><i>Numerical Methods in Engineering with Python 3</i> by Jaan Kiusalaas</center>

---
<center><h2>Question 12</h2></center>
Solve the following equations with the Gauss-Seidel method: 

$$
\begin{bmatrix}
12 & -2 & 3 & 1 \\
-2 & 15 & 6 & -3 \\
1 & 6 & 20 & -4 \\
0 & -3 & 2 & 9 \\
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
x_4 \\
\end{bmatrix} = 
\begin{bmatrix}
0 \\
0 \\
20 \\
0 \\
\end{bmatrix}
$$

In [14]:
# Solve for x using the Gauss-Seidel Method
A = array([[ 12, -2,  3,  1 ],
           [ -2, 15,  6, -3 ],
           [  1,  6, 20, -4 ],
           [  0, -3,  2,  9 ] ], float)
b = array( [ 0, 0, 20, 0 ], float)

# Initial Approximation:: set x=0
x = array([0, 0, 0, 0], float )

## Define functions for iterative technique
def gaussSeidel(iterEqs, x,tol = 1.0e-9):
    omega = 1  # no relaxation
    k = 10
    p = 1
    imax = 100
    for i in range(1, imax):
        xOld = x.copy()
        x = iterEqs(x, omega)
        dx = x - xOld
        dx = sqrt( dx @ dx )
        if dx < tol:
            return x,i,omega
        # Compute relaxation factor after k+p iterations
        if i == k: dx1 = dx
        if i == k + p:
            dx2 = dx
            omega = 2.0/(1.0 + math.sqrt(1.0 - (dx2/dx1)**(1.0/p)))
    return x,i,omega
def iterEqs(x, omega = 1): 
    gs_update = ( b - ( A @ x - diag(A) * x ) ) / diag(A) 
    return  omega * gs_update + (1-omega) * x

# Solution
x, numIter, omega = gaussSeidel(iterEqs, x)
x_orig = x.copy()

# Results
print("\nNumber of equations ==> ",A.shape[0])
print("\nTotal Number of Iterations =", numIter)
# print("\nRelaxation factor =", omega)
print("\nNorm of the Residual Vector::\n", np.linalg.norm(A @ x - b))
print("\nThe solution Vector x =\n", x)
print("\nVerify A x = b:\n\t→A x:", np.round((A @ x), 1), "\n\t→b:", b,"\n")


Number of equations ==>  4

Total Number of Iterations = 29

Norm of the Residual Vector::
 8.13460847711607e-09

The solution Vector x =
 [-0.3344 -0.5722  1.1013 -0.4355]

Verify A x = b:
	→A x: [ 0.  0. 20. -0.] 
	→b: [ 0.  0. 20.  0.] 



<br>
Repeat the above using the $Conjugate Gradient Method$
<br><br>

In [6]:
# Solve for x using the Conjugate Gradient Method
A = array([[ 12, -2,  3,  1 ],
           [ -2, 15,  6, -3 ],
           [  1,  6, 20, -4 ],
           [  0, -3,  2,  9 ] ], float)
b = array( [ 0, 0, 20, 0 ], float)

# Initial Approximation:: set x=0
x = array([0, 0, 0, 0], float )

## Define functions for iterative technique
def conjGrad(Av,x,b,tol=1.0e-9):
    n = len(b)
    r = b - Av(x)
    s = r.copy()
    for i in range(n):
        u = Av(s)
        alpha = np.dot(s,r)/np.dot(s,u)
        x = x + alpha*s
        r = b - Av(x)
        if(math.sqrt(np.dot(r,r))) < tol:
            break
        else:
            beta = -np.dot(r,u)/np.dot(s,u)
            s = r + beta*s
    return x,i
def Ax(v):
    n = len(v)
    Ax = np.zeros(n)
    Ax[0] = 12.0*v[0] - 2.0*v[1] + 3.0*v[2] + v[3]
    Ax[1:n-2] = -2.0*v[0:n-3] + 15.0*v[1:n-2] + 6.0*v[2:n-1] - 3.0*v[3:n]
    Ax[2:n-1] = v[0:n-3] + 6.0*v[1:n-2] + 20.0*v[2:n-1] - 4.0*v[3:n]
    Ax[n-1] = -3.0*v[n-3] + 2.0*v[n-2] + 9.0*v[n-1]
    if np.isnan(Ax).any():
        print('NaN found!')
    return Ax

# Solution
x, numIter = conjGrad(Ax,x,b,tol=1.0e-9)
if np.isnan(x).any() or (A@x).all() != b.all():
    print('\n--->>> The Conjugate Gradient Method cannot be applied here! <<<---\n')
    print('\tA sparse matrix with symmetry is required.\n')
    print("Problematic Solution Vector x =\n", x)
    print("\nVerify A x ≠ b:\n\t→A x:", np.round((A @ x), 1), "\n\t→b:", b,"\n")
else:
    # Results
    print("Number of equations ==> ",A.shape[0])
    print("\nTotal Number of Iterations = ", numIter)
    print("\nNorm of the Residual Vector = ", np.linalg.norm(A @ x - b))
    print("\nThe Solution Vector x =\n", x)
    print("\nVerify A x = b:\n\t→A x:", np.round((A @ x), 1), "\n\t→b:", b,"\n")


--->>> The Conjugate Gradient Method cannot be applied here! <<<---

	A sparse matrix with symmetry is required.

Problematic Solution Vector x =
 [-0.3539 -0.5781  1.1165 -0.4714]

Verify A x ≠ b:
	→A x: [-0.2  0.1 20.4 -0.3] 
	→b: [ 0.  0. 20.  0.] 



---
<center><h2>Question 13</h2></center>
Use the Gauss-Seidel method with relaxation to solve <b>$Ax = b$</b>, where

$$
A = 
\begin{bmatrix}
4 & -1 & 0 & 0 \\
-1 & 4 & -1 & 0 \\
0 & -1 & 4 & -1 \\
0 & 0 & -1 & 3 \\
\end{bmatrix}    
b = 
\begin{bmatrix}
15 \\
10 \\
10 \\
10 \\
\end{bmatrix}
$$

Take $x_i = \frac{b_i}{A_{ii}}$ as the starting vector, and use $\omega = 1.1$ for the relaxation factor.

In [7]:
# Solve for x using the Gauss-Seidel Method & relaxation omega = 1.1
A = array( [[ 4, -1,  0,  0 ], 
            [-1,  4, -1,  0 ], 
            [ 0, -1,  4, -1 ], 
            [ 0,  0, -1,  3 ]], float) 
b = array( [ 15, 10, 10, 10 ], float)

# Initial Approximation:: set x=b/A
x = np.zeros(A.shape[0])
for i in range(len(b)):
    x[i] = b[i]/A[i][i]
print('\n--->> The Starting Vector x is', x, '\n')

# Define functions for iterative technique
def gaussSeidel(iterEqs, x,tol = 1.0e-9):
    omega = 1.1  # <<-- extrapolation
    k = 10
    p = 1
    imax = 100
    for i in range(1, imax):
        xOld = x.copy()
        x = iterEqs(x, omega)
        dx = x - xOld
        dx = sqrt( dx @ dx )
        if dx < tol:
            return x,i,omega
    return x,i,omega
def iterEqs(x, omega = 1): 
    gs_update = ( b - ( A @ x - diag(A) * x ) ) / diag(A) 
    return  omega * gs_update + (1-omega) * x 

# Solution
n = A.shape[0]
x, numIter, omega = gaussSeidel(iterEqs, x)
# x_orig = x.copy()

# Results
print("Number of equations ==> ",n)
print("\nTotal Number of Iterations =", numIter)
print("\nRelaxation factor =", omega)
print("\nNorm of the Residual Vector::\n", np.linalg.norm(A @ x - b))
print("\nThe solution Vector x =\n", x)
print("\nVerify A x = b:\n\t→A x:", np.round((A @ x), 1), "\n\t→b:", b,"\n")


--->> The Starting Vector x is [3.75   2.5    2.5    3.3333] 

Number of equations ==>  4

Total Number of Iterations = 33

Relaxation factor = 1.1

Norm of the Residual Vector::
 1.9329380228492633e-09

The solution Vector x =
 [5. 5. 5. 5.]

Verify A x = b:
	→A x: [15. 10. 10. 10.] 
	→b: [15. 10. 10. 10.] 



<br>
Repeat the above using the $Conjugate Gradient Method$
<br><br>

In [8]:
# Solve for x using the Conjugate Gradient Method & relaxation omega = 1.1
A = array( [[ 4, -1,  0,  0 ], 
            [-1,  4, -1,  0 ], 
            [ 0, -1,  4, -1 ], 
            [ 0,  0, -1,  3 ]], float) 
b = array( [ 15, 10, 10, 10 ], float)

# Initial Approximation:: set x=b/A
x = np.zeros(A.shape[0])
for i in range(len(b)):
    x[i] = b[i]/A[i][i]
print('\n--->> The Starting Vector x is', x, '\n')

## Define functions for iterative technique
def conjGrad(Av,x,b,tol=1.0e-9):
    n = len(b)
    r = b - Av(x)
    s = r.copy()
    for i in range(n):
        u = Av(s)
        alpha = np.dot(s,r)/np.dot(s,u)
        x = x + alpha*s
        r = b - Av(x)
        if(math.sqrt(np.dot(r,r))) < tol:
            break
        else:
            beta = -np.dot(r,u)/np.dot(s,u)
            s = r + beta*s
    return x,i
def Ax(v):
    n = len(v)
    Ax = np.zeros(n)
    Ax[0] = 4.0*v[0] - v[1]
    Ax[1:n-1] = -v[0:n-2] + 4.0*v[1:n-1] - v[2:n]
    Ax[n-1] = -v[n-2] + 3.0*v[n-1]
    if np.isnan(Ax).any():
        print('NaN found!')
    return Ax

# Solution
x, numIter = conjGrad(Ax,x,b,tol=1.0e-9)
if np.isnan(x).any() or (A@x).all() != b.all():
    print('\n--->>> The Conjugate Gradient Method cannot be applied here! <<<---\n')
    print('\tA sparse matrix with symmetry is required.\n')
    print("Problematic Solution Vector x =\n", x)
    print("\nVerify A x ≠ b:\n\t→A x:", np.round((A @ x), 1), "\n\t→b:", b,"\n")
else:
    # Results
    print("Number of equations ==> ",A.shape[0])
    print("\nTotal Number of Iterations = ", numIter)
    print("\nNorm of the Residual Vector = ", np.linalg.norm(A @ x - b))
    print("\nThe Solution Vector x =\n", x)
    print("\nVerify A x = b:\n\t→A x:", np.round((A @ x), 1), "\n\t→b:", b,"\n")


--->> The Starting Vector x is [3.75   2.5    2.5    3.3333] 

Number of equations ==>  4

Total Number of Iterations =  3

Norm of the Residual Vector =  0.0

The Solution Vector x =
 [5. 5. 5. 5.]

Verify A x = b:
	→A x: [15. 10. 10. 10.] 
	→b: [15. 10. 10. 10.] 



---
<center><h2>Question 16</h2></center>
Write a program for solving <b>$Ax = b$</b> by the Gauss-Seidel method based on the function $\color{magenta}{\text{gaussSeidel}}$. Input should consist of the matrix <b>$A$</b> and the vector <b>$b$</b>. Test the program with

$$
A = 
\begin{bmatrix}
3 & -2 & 1 & 0 & 0 & 1 \\
-2 & 4 & -2 & 1 & 0 & 0 \\
1 & -2 & 4 & -2 & 1 & 0 \\
0 & 1 & -2 & 4 & -2 & 1 \\
0 & 0 & 1 & -2 & 4 & -2 \\
1 & 0 & 0 & 1 & -2 & 3 \\
\end{bmatrix}    
b = 
\begin{bmatrix}
10 \\
-8 \\
10 \\
10 \\
-8 \\
10 \\
\end{bmatrix}
$$

Note that <b>$A$</b> is not diagonally dominant, but this does not necessarily preclude convergence.

In [65]:
# Solve for x using the Gauss-Seidel Method
## Define function to re-create A according to input length 'n'
def banded_matrix(n):
    A0 = diags([1,-2,4,-2,-1],[-2,-1,0,1,2], shape=(n,n)).toarray()
    A0[0][0] -= 1  # adjust UL corner
    A0[n-1][n-1] -= 1  # adjust LR corner
    A0[0][n-1] = 1  # adjust LL corner
    A0[n-1][0] = 1  # adjust UR corner
    return A0
b = array( [10, -8, 10, 10, -8, 10 ], float)
    
## Define functions for iterative technique
def gaussSeidel(a, B):
    omega = 1.0  # no relaxation
    k = 10
    p = 1
    imax = 2000  # max iterations
    x = zeros(len(B))  # initial approximation:: set x=0
    for i in range(1, imax):
        xOld = x.copy()
        for j in range(len(B)):
            gs_update = ( B - ( a @ x - diag(a) * x ) ) / diag(a) 
        x = omega * gs_update + (1-omega) * x
        dx = math.sqrt(np.dot(x-xOld,x-xOld))
        if dx < 1.0e-9: return x,i,omega
        # Compute relaxation factor after k+p iterations
        if i == k: dx1 = dx
        if i == k + p:
            dx2 = dx
            omega = 2.0/(1.0 + math.sqrt(1.0 - (dx2/dx1)**(1.0/p)))
    print(f'\n\n--->>> Gauss-Seidel failed to converge within', \
          f'{imax} iterations <<<---\n')
    # Results
    print("\nNumber of equations ==> ", a.shape[0])
    print("\nTotal Number of Iterations = ", i)
    print("\nNorm of the Residual Vector = ", np.linalg.norm(a @ x - B))
    return x,i,omega

# Solution
x = np.zeros(len(b))
A = banded_matrix(len(b))
x, numIter, omega = gaussSeidel(A, b)

# Results
if np.isnan(x).any() or (A@x).all() != b.all():
    print("\nProblematic Solution Vector x =\n", x)
    print("\nVerify A x ≠ b:\n\t→A x:", np.round((A @ x), 1), "\n\t→b:", b,"\n")
    print('Suspect that A is positive-indefinite and x is a saddle point?\n')
else:
    # Results
    print("\nThe Solution Vector x =\n", x)
    print("\nVerify A x = b:\n\t→A x:", np.round((A @ x), 1), "\n\t→b:", b,"\n")



--->>> Gauss-Seidel failed to converge within 2000 iterations <<<---


Number of equations ==>  6

Total Number of Iterations =  1999

Norm of the Residual Vector =  nan

Problematic Solution Vector x =
 [nan nan nan nan nan nan]

Verify A x ≠ b:
	→A x: [nan nan nan nan nan nan] 
	→b: [10. -8. 10. 10. -8. 10.] 

Suspect that A is positive-indefinite and x is a saddle point?



<br>
Repeat the above using the $Conjugate Gradient Method$
<br><br>

In [18]:
# Solve for x using the Conjugate Gradient Method
A = array([[ 3, -2,  1,  0,  0,  1 ],
           [-2,  4, -2,  1,  0,  0 ],
           [ 1, -2,  4, -2,  1,  0 ],
           [ 0,  1, -2,  4, -2,  1 ],
           [ 0,  0,  1, -2,  4, -2 ],
           [ 1,  0,  0,  1, -2,  3 ]], float) # note: bands are neg
b = array( [10, -8, 10, 10, -8, 10 ], float)

## Define functions for iterative technique
def conjGrad(a, B):
    '''
    Function accepts a Transformation Matrix "a" with Output Vector "B"
    when applied to some Solution Vector "x". Function finds x.
    Author's Note: 'pred' == 'approximated'
    DEFINITIONS::
    f' :: gradient; vectors are direction (steepest increase of f), magnitude (rate of increase)
    r :: residual vector, stores direction of steepest descent btwn pred-x and true-x; 
            r=B-a*x (keep implementing eq to update a*r product btwn alpha & new_r to avoid roundoff)
    new_r :: next prediction for r based on previous r; new_r = prev_r-alpha*a*r
    alpha :: step size, minimizes f per iteration by setting f'=(-r); alpha=r.T*r/r.T(a*r)
    pred_x :: next prediction for x based on previous x; pred_x=prev_x+alpha*r
    '''
    n = len(B)
    x = np.zeros(n)  # initial approximation of the solution:: set x=0
    r = np.zeros(n)  # initial residual vector
    r = B - np.dot(a,x)
    s = r.copy()  # initial search vector
    r_old = np.dot(r.T,r)
    for i in range(n):
        u = np.dot(a,s) # unit vectors
        alpha = r_old / np.dot(s.T,u)
        x += np.dot(alpha,s) # iterated solution
        r -= np.dot(alpha,u) # updated residual
        r_new = np.dot(r.T,r)
        # Check whether converging
        if(math.sqrt(np.dot(r,r))) < 1.0e-9: break
        s = r + (r_new/r_old)*s # updated search direction
        r_old = r_new
    return x,i

# Solution
x, numIter = conjGrad(A,b)

# Results
if np.isnan(x).any() or (A@x).all() != b.all():
    print('\n--->>> The Conjugate Gradient Method cannot be applied here! <<<---\n')
    print('\tA sparse matrix with symmetry is required.\n')
    print("Problematic Solution Vector x =\n", x)
    print("\nVerify A x ≠ b:\n\t→A x:", np.round((A @ x), 1), "\n\t→b:", b,"\n")
else:
    print("\nNumber of equations ==> ",A.shape[0])
    print("\nTotal Number of Iterations = ", numIter)
    print("\nNorm of the Residual Vector = ", np.linalg.norm(A @ x - b))
    print("\nThe Solution Vector x =\n", x)
    print("\nVerify A x = b:\n\t→A x:", np.round((A @ x), 1), "\n\t→b:", b,"\n")


Number of equations ==>  6

Total Number of Iterations =  2

Norm of the Residual Vector =  7.793734002518813e-15

The Solution Vector x =
 [ 1.3 -0.3  4.2  4.2 -0.3  1.3]

Verify A x = b:
	→A x: [10. -8. 10. 10. -8. 10.] 
	→b: [10. -8. 10. 10. -8. 10.] 



---
<center><h2>Question 17</h2></center>
Modify the program in Example 2.17 (Gauss-Seidel method) so that it will solve the following equations:

$$
\begin{bmatrix}
4 & -1 & 0 & 0 & ... & 0 & 0 & 0 & 1 \\
-1 & 4 & -1 & 0 & ... & 0 & 0 & 0 & 0 \\
0 & -1 & 4 & -1 & ... & 0 & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & 0 & ... & -1 & 4 & -1 & 0 \\
0 & 0 & 0 & 0 & ... & 0 & -1 & 4 & -1 \\
1 & 0 & 0 & 0 & ... & 0 & 0 & -1 & 4 \\
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
\vdots \\
x_{n-2} \\
x_{n-1} \\
x_n \\
\end{bmatrix} = 
\begin{bmatrix}
0 \\
0 \\
0 \\
\vdots \\
0 \\
0 \\
100 \\
\end{bmatrix}
$$

Run the program with $n = 20$ and compare the number of iterations with Example 2.17.

In [62]:
# Modify textbook models to solve Example 2.17 using Gauss-Seidel Method
## Define function to re-create A according to input length 'n'
def banded_matrix(n):
    A0 = diags([-1,4,-1],[-1,0,1], shape=(n,n)).toarray()
    A0[0][n-1] = 1
    A0[n-1][0] = 1
    b0 = np.zeros(n)
    b0[n-1] = 100
    return A0, b0

## Define function for Gauss-Seidel iterative technique
def gaussSeidel(a, B):
    omega = 1.0  # no relaxation
    k = 10
    p = 1
    imax = 2000  # max iterations
    x = zeros(len(B))  # initial approximation:: set x=0
    for i in range(1, imax):
        xOld = x.copy()
        for j in range(len(B)):
            gs_update = ( B - ( a @ x - diag(a) * x ) ) / diag(a) 
        x = omega * gs_update + (1-omega) * x
        dx = math.sqrt(np.dot(x-xOld,x-xOld))
        if dx < 1.0e-9: return x,i,omega
        # Compute relaxation factor after k+p iterations
        if i == k: dx1 = dx
        if i == k + p:
            dx2 = dx
            omega = 2.0/(1.0 + math.sqrt(1.0 - (dx2/dx1)**(1.0/p)))
    print(f'\n\n--->>> Gauss-Seidel failed to converge within', \
          f'{imax} iterations <<<---\n')
    # Results
    print("\nNumber of equations ==> ", a.shape[0])
    print("\nTotal Number of Iterations = ", i)
    print("\nNorm of the Residual Vector = ", np.linalg.norm(a @ x - B))
    return x,i,omega

# Solution
n = eval(input("\nEnter the number of equations ==> "))
x = np.zeros(n)
A, b = banded_matrix(n)
x, numIter, omega = gaussSeidel(A, b)

# Results
print("\nNumber of equations ==> ",A.shape[0])
print("\nTotal Number of Iterations =", numIter)
print("\nRelaxation factor =", omega)
print("\nNorm of the Residual Vector::\n", np.linalg.norm(A @ x - b))
print("\nThe solution Vector x =\n", x)
print("\nVerify A x = b:\n\t→A x:", np.round((A @ x), 1), "\n\t→b:", b,"\n")


Enter the number of equations ==>  20



Number of equations ==>  20

Total Number of Iterations = 63

Relaxation factor = 1.1654978831268998

Norm of the Residual Vector::
 2.302717522444513e-09

The solution Vector x =
 [-7.7350e+00 -2.0726e+00 -5.5535e-01 -1.4881e-01 -3.9872e-02 -1.0683e-02
 -2.8616e-03 -7.6311e-04 -1.9078e-04  0.0000e+00  1.9078e-04  7.6311e-04
  2.8616e-03  1.0683e-02  3.9872e-02  1.4881e-01  5.5535e-01  2.0726e+00
  7.7350e+00  2.8868e+01]

Verify A x = b:
	→A x: [  0.  -0.   0.  -0.   0.  -0.   0.  -0.   0.   0.  -0.   0.  -0.   0.
  -0.   0.  -0.   0.  -0. 100.] 
	→b: [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0. 100.] 



<br>
Repeat the above using the $Conjugate Gradient Method$
<br><br>

In [63]:
# Modify textbook models to solve Example 2.17 using Conjugate Gradient Method
## Define function to re-create A according to input length 'n'
def banded_matrix(n):
    A0 = diags([-1,4,-1],[-1,0,1], shape=(n,n)).toarray()
    A0[0][n-1] = 1
    A0[n-1][0] = 1
    b0 = np.zeros(n)
    b0[n-1] = 100
    return A0, b0

## Define functions for iterative technique
def conjGrad(a, B):
    n = len(B)
    x = np.zeros(n)  # initial approximation:: set x=0
    r = np.zeros(n)  # initial residual vector
    r = B - np.dot(a,x)
    s = r.copy()  # initial search vector
    r_old = np.dot(r.T,r)
    for i in range(n):
        u = np.dot(a,s) # unit vectors
        alpha = r_old / np.dot(s.T,u)
        x += np.dot(alpha,s) # iterated solution
        r -= np.dot(alpha,u) # updated residual
        r_new = np.dot(r.T,r)
        # Check whether converging
        if(math.sqrt(np.dot(r,r))) < 1.0e-9: break
        s = r + (r_new/r_old)*s # updated search direction
        r_old = r_new
    return x,i

# Solution
n = eval(input("\nEnter the number of equations ==> "))
x = np.zeros(n)
A, b = banded_matrix(n)
x, numIter = conjGrad(A, b)

# Results
if np.isnan(x).any() or (A@x).all() != b.all():
    print('\n--->>> The Conjugate Gradient Method cannot be applied here! <<<---\n')
    print('\tA sparse matrix with symmetry is required.\n')
    print("Problematic Solution Vector x =\n", x)
    print("\nVerify A x ≠ b:\n\t→A x:", np.round((A @ x), 1), "\n\t→b:", b,"\n")
else:
    print("\nNumber of equations ==> ",A.shape[0])
    print("\nTotal Number of Iterations = ", numIter)
    print("\nNorm of the Residual Vector = ", np.linalg.norm(A @ x - b))
    print("\nThe Solution Vector x =\n", x)
    print("\nVerify A x = b:\n\t→A x:", np.round((A @ x), 1), "\n\t→b:", b,"\n")


Enter the number of equations ==>  20



Number of equations ==>  20

Total Number of Iterations =  9

Norm of the Residual Vector =  1.5076212769453322e-14

The Solution Vector x =
 [-7.7350e+00 -2.0726e+00 -5.5535e-01 -1.4881e-01 -3.9872e-02 -1.0683e-02
 -2.8616e-03 -7.6311e-04 -1.9078e-04  0.0000e+00  1.9078e-04  7.6311e-04
  2.8616e-03  1.0683e-02  3.9872e-02  1.4881e-01  5.5535e-01  2.0726e+00
  7.7350e+00  2.8868e+01]

Verify A x = b:
	→A x: [  0.  -0.   0.   0.   0.  -0.   0.   0.  -0.   0.   0.   0.  -0.   0.
  -0.  -0.   0.   0.   0. 100.] 
	→b: [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0. 100.] 



--- 
--- 

<center><h2>Analysis</h2></center><br><br>

| Iterative Method <br> Performs Best for :: | Gauss-Seidel | Conjugate Gradient |
| --- | --- | --- |
| Assymetric Matrices | x |  |
| Computational Time |   | x |
| Large Matrices | x | x |
| Non-Diagonally Dominant|   | x |